In [0]:
# Inferential Statistics vs Big Data Analytics
## NYC Yellow Taxi Trips (2022–2024)

from pyspark.sql.functions import *
import pandas as pd
import matplotlib.pyplot as plt


Charger le dataset SAMPLE


In [0]:
sample_path = "/Volumes/workspace/default/filestore/yellowtaxisample1pct_hybrid_stratified.csv"

df_sample = pd.read_csv(sample_path)
df_sample.shape
